# We will be creating the folds for one-point tracking, six-point tracking, and all-points tracking!

In [2]:
# first track all points

K = 5

import mediapipe as mp 
from PIL import Image as im 
import mediapipe as mp

def hand_locations(frame, min_detection_confidence = 0.5, min_tracking_confidence = 0.5): 
    hands = mp.solutions.hands.Hands(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) # MAKE SURE THIS IS ALL GOOD 
    results = hands.process(frame.astype('uint8'))
    X_locations = [0] * 42 # use 0 as default if the class is not there 
    Y_locations = [0] * 42 
    Z_locations = [0] * 42
    x = y = z = 0 
    if results.multi_hand_landmarks:
        for hand, hand_landmark in enumerate(results.multi_hand_landmarks):
            for i in range(0, 21):
                landmark = hand_landmark.landmark[i]
                X_locations[x] = landmark.x
                Y_locations[y] = landmark.y
                Z_locations[z] = landmark.z
                x += 1; y += 1; z += 1; 
    locations = np.concatenate([X_locations, Y_locations, Z_locations])
    hands.close()
    return locations 

# time to actually do calibration. 

SECONDS_TO_DETECT = 2 # in seconds

import os, cv2
from tqdm import tqdm 
import numpy as np

ARMFLAPPING_VIDEOS = []
CONTROL_VIDEOS = []
ARMFLAPPING_FPS = [] # store the FPS of all armflapping videos 
CONTROL_FPS = [] # store the FPS of all control videos 

# load these same file names 
import pickle
with open("FILE_NAMES.pkl", 'rb') as f: 
    ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES = pickle.load(f)

for video_name in tqdm(ARMFLAPPING_FILE_NAMES, desc = "armflapping_videos"): 
    try: 
        os.mkdir("behavior_data/armflapping/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/armflapping/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 

        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break  
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image) 

            cv2.imwrite("behavior_data/armflapping/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            i += 1 

        ARMFLAPPING_VIDEOS.append(FRAMES)
        ARMFLAPPING_FPS.append(frame_rate)

    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

for video_name in tqdm(CONTROL_FILE_NAMES, desc = "control_videos"): 
    try:
        os.mkdir("behavior_data/control/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/control/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 
        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break 
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image)

            cv2.imwrite("behavior_data/control/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            
            i += 1
        CONTROL_VIDEOS.append(FRAMES)
        CONTROL_FPS.append(frame_rate)
        
    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS = [], []

for FRAMES, file_name in tqdm(zip(CONTROL_VIDEOS, CONTROL_FILE_NAMES)):  
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/control/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    CONTROL_LOCATIONS.append(locs)


# get the locations of all of the videos 

for FRAMES, file_name in tqdm(zip(ARMFLAPPING_VIDEOS,ARMFLAPPING_FILE_NAMES)) :
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/armflapping/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    ARMFLAPPING_LOCATIONS.append(locs)

N = min([len(locs) for locs in [ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS]])
ARMFLAPPING_LOCATIONS = ARMFLAPPING_LOCATIONS[:N]
CONTROL_LOCATIONS = CONTROL_LOCATIONS[:N]
ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES[:N])
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES[:N])

# we can create a padding function in order to pad 
def pad(locations, maxlen = 90, padding = "post", truncating = "post"): 
    new_locations = locations.tolist() 
    empty_row = np.zeros((1, 126))
    for i, video in tqdm(enumerate(new_locations)): 
        if len(video) < maxlen:  
            for new_row in range(maxlen - len(video)): 
                if padding == "post": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([new_locations[i], empty_row])
                if padding == "pre": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([empty_row, new_locations[i]])

        if len(video) > maxlen: 
            if truncating == "post": 
                new_locations[i] = new_locations[i][:maxlen]
            elif truncating == "pre": 
                new_locations[i] = new_locations[i][len(video) - maxlen : ]
    return np.array(new_locations)

padded_armflapping_locations = ARMFLAPPING_LOCATIONS
padded_control_locations = CONTROL_LOCATIONS
padded_armflapping_locations = pad(padded_armflapping_locations, maxlen = 90)
padded_control_locations = pad(padded_control_locations, maxlen = 90)
print(padded_control_locations.shape, padded_armflapping_locations.shape)
assert padded_armflapping_locations.shape == padded_control_locations.shape 

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES)
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES)

"""
# first shuffle 
N = padded_armflapping_locations.shape[0]
perm = np.random.permutation(N)
padded_armflapping_locations = padded_armflapping_locations[perm]
padded_control_locations = padded_control_locations[perm]
ARMFLAPPING_FILE_NAMES = ARMFLAPPING_FILE_NAMES[perm]
CONTROL_FILE_NAMES = CONTROL_FILE_NAMES[perm]
"""

# then balance each fold (still should do this) 
padded_armflapping_locations = np.array_split(padded_armflapping_locations, K)
padded_control_locations = np.array_split(padded_control_locations, K)
ARMFLAPPING_FILE_NAMES = np.array_split(ARMFLAPPING_FILE_NAMES, K)
CONTROL_FILE_NAMES = np.array_split(CONTROL_FILE_NAMES, K)

X_splits = []
y_splits = []

for i in range(K): 
    X_splits.append(np.concatenate([padded_armflapping_locations[i], padded_control_locations[i]]))
    y_splits.append(np.concatenate([np.ones((len(padded_armflapping_locations[i]), 1)), np.zeros((len(padded_control_locations[i]), 1))]))

import pickle

for i, (X_split, y_split, A_FILE_NAMES, C_FILE_NAMES) in enumerate(zip(X_splits, y_splits, ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES)): 
    with open(f"all_points_folds/split{i+1}", 'wb') as f: 
        pickle.dump((X_split, y_split), f)
    

with open(f"all_point_folds_seeds", "wb") as f: 
    pickle.dump((padded_armflapping_locations, ARMFLAPPING_FILE_NAMES, padded_control_locations, CONTROL_FILE_NAMES), f)

control_videos: 100%|██████████| 50/50 [00:23<00:00,  2.13it/s]
0it [00:00, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
50it [01:42,  2.05s/it]
50it [01:49,  2.20s/it]
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_12388/4098782028.py:127: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_12388/4098782028.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)
50it [00

(50, 90, 126) (50, 90, 126)


In [4]:
# next, track six points

text here to make sure you don't run this cell again. just look in six_point_folds_seeds for all data required.  

K = 5

import mediapipe as mp 
from PIL import Image as im 
import numpy as np

np.random.seed(42) # set a random seed 

def hand_locations(frame, min_detection_confidence = 0.5, min_tracking_confidence = 0.5): 
    """Only give 6 landmarks"""

    hands = mp.solutions.hands.Hands(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) # MAKE SURE THIS IS ALL GOOD 
    results = hands.process(frame.astype('uint8'))
    X_locations = [0] * 12
    Y_locations = [0] * 12
    Z_locations = [0] * 12

    if results.multi_hand_landmarks:
        x = y = z = 0 
        for hand, hand_landmark in enumerate(results.multi_hand_landmarks):
            for i in range(0, 21):
                if i not in [0, 4, 8, 12, 16, 20]: continue 
                landmark = hand_landmark.landmark[i]
                X_locations[x] = landmark.x
                Y_locations[y] = landmark.y 
                Z_locations[z] = landmark.z
                x += 1; y += 1; z +=1; 
            
    hands.close()
    return np.concatenate([X_locations, Y_locations, Z_locations]) 

# time to actually do calibration. 

SECONDS_TO_DETECT = 2 # in seconds

import os, cv2
from tqdm import tqdm 
import numpy as np

ARMFLAPPING_VIDEOS = []
CONTROL_VIDEOS = []
ARMFLAPPING_FPS = [] # store the FPS of all armflapping videos 
CONTROL_FPS = [] # store the FPS of all control videos 

import pickle
with open("FILE_NAMES.pkl", 'rb') as f: 
    ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES = pickle.load(f)
for video_name in tqdm(ARMFLAPPING_FILE_NAMES, desc = "armflapping_videos"): 
    try: 
        os.mkdir("behavior_data/armflapping/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/armflapping/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 

        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break  
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image) 

            cv2.imwrite("behavior_data/armflapping/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            i += 1 

        ARMFLAPPING_VIDEOS.append(FRAMES)
        ARMFLAPPING_FPS.append(frame_rate)

    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

for video_name in tqdm(CONTROL_FILE_NAMES, desc = "control_videos"): 
    try:
        os.mkdir("behavior_data/control/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/control/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 
        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break 
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image)

            cv2.imwrite("behavior_data/control/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            
            i += 1
        CONTROL_VIDEOS.append(FRAMES)
        CONTROL_FPS.append(frame_rate)
        
    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS = [], []

for FRAMES, file_name in tqdm(zip(CONTROL_VIDEOS, CONTROL_FILE_NAMES)):  
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/control/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    CONTROL_LOCATIONS.append(locs)


# get the locations of all of the videos 

for FRAMES, file_name in tqdm(zip(ARMFLAPPING_VIDEOS,ARMFLAPPING_FILE_NAMES)) :
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/armflapping/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    ARMFLAPPING_LOCATIONS.append(locs)

N = min([len(locs) for locs in [ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS]])
ARMFLAPPING_LOCATIONS = ARMFLAPPING_LOCATIONS[:N]
CONTROL_LOCATIONS = CONTROL_LOCATIONS[:N]
ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES[:N])
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES[:N])

# we can create a padding function in order to pad 
def pad(locations, maxlen = 90, padding = "post", truncating = "post"): 
    new_locations = locations.tolist() 
    empty_row = np.zeros((1, 36))
    for i, video in tqdm(enumerate(new_locations)): 
        if len(video) < maxlen:  
            for new_row in range(maxlen - len(video)): 
                if padding == "post": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([new_locations[i], empty_row])
                if padding == "pre": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([empty_row, new_locations[i]])

        if len(video) > maxlen: 
            if truncating == "post": 
                new_locations[i] = new_locations[i][:maxlen]
            elif truncating == "pre": 
                new_locations[i] = new_locations[i][len(video) - maxlen : ]
    return np.array(new_locations)

padded_armflapping_locations = ARMFLAPPING_LOCATIONS
padded_control_locations = CONTROL_LOCATIONS
padded_armflapping_locations = pad(padded_armflapping_locations, maxlen = 90)
padded_control_locations = pad(padded_control_locations, maxlen = 90)
print(padded_control_locations.shape, padded_armflapping_locations.shape)
assert padded_armflapping_locations.shape == padded_control_locations.shape 

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES)
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES)

# first shuffle 
N = padded_armflapping_locations.shape[0]
perm = np.random.permutation(N)
padded_armflapping_locations = padded_armflapping_locations[perm]
padded_control_locations = padded_control_locations[perm]
ARMFLAPPING_FILE_NAMES = ARMFLAPPING_FILE_NAMES[perm]
CONTROL_FILE_NAMES = CONTROL_FILE_NAMES[perm]

# then balance each fold 
padded_armflapping_locations = np.array_split(padded_armflapping_locations, K)
padded_control_locations = np.array_split(padded_control_locations, K)
ARMFLAPPING_FILE_NAMES = np.array_split(ARMFLAPPING_FILE_NAMES, K)
CONTROL_FILE_NAMES = np.array_split(CONTROL_FILE_NAMES, K)

X_splits = []
y_splits = []

for i in range(K): 
    X_splits.append(np.concatenate([padded_armflapping_locations[i], padded_control_locations[i]]))
    y_splits.append(np.concatenate([np.ones((len(padded_armflapping_locations[i]), 1)), np.zeros((len(padded_control_locations[i]), 1))]))

import pickle

for i, (X_split, y_split, A_FILE_NAMES, C_FILE_NAMES) in enumerate(zip(X_splits, y_splits, ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES)): 
    with open(f"six_point_folds/split{i+1}", 'wb') as f: 
        pickle.dump((X_split, y_split), f)
    

with open(f"six_point_folds_seeds", "wb") as f: 
    pickle.dump((padded_armflapping_locations, ARMFLAPPING_FILE_NAMES, padded_control_locations, CONTROL_FILE_NAMES), f)

armflapping_videos:  12%|█▏        | 13/109 [00:03<00:11,  8.42it/s]OpenCV: Couldn't read video stream from file "behavior_data/armflapping/.DS_Store"
[ERROR:0] global /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/videoio/src/cap.cpp (162) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.3) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): behavior_data/armflapping/.DS_Store in function 'icvExtractPattern'


armflapping_videos:  14%|█▍        | 15/109 [00:03<00:12,  7.55it/s]

float division by zero
failed on .DS_Store


armflapping_videos:  56%|█████▌    | 61/109 [00:17<00:05,  8.86it/s]

float division by zero
failed on .mp4


control_videos:  19%|█▉        | 12/62 [00:05<00:24,  2.02it/s]OpenCV: Couldn't read video stream from file "behavior_data/control/.DS_Store"
[ERROR:0] global /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/videoio/src/cap.cpp (162) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.3) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): behavior_data/control/.DS_Store in function 'icvExtractPattern'




float division by zero
failed on .DS_Store


control_videos: 100%|██████████| 62/62 [00:22<00:00,  2.79it/s]
50it [01:42,  2.05s/it]
97it [03:24,  2.11s/it]
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_4906/1597820171.py:134: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_4906/1597820171.py:135: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)
50it [00:00, 16350.79it/s]
50it [00:00, 11582.64it/s]

(50, 90, 36) (50, 90, 36)


In [4]:
# finally, only track one point

K = 5

import mediapipe as mp 
from PIL import Image as im 
import mediapipe as mp


def hand_locations(frame, min_detection_confidence = 0.5, min_tracking_confidence = 0.5): 
    """Only give the 0th landmark"""

    hands = mp.solutions.hands.Hands(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence)  
    results = hands.process(frame.astype('uint8'))
    X_locations = [0] * 2
    Y_locations = [0] * 2
    Z_locations = [0] * 2
    if results.multi_hand_landmarks:
        for hand, hand_landmark in enumerate(results.multi_hand_landmarks):
            for i in range(0, 21):
                landmark = hand_landmark.landmark[i]
                X_locations[hand] = landmark.x
                Y_locations[hand] = landmark.y
                Z_locations[hand] = landmark.z
                break # take only the first landmark
            
    hands.close()
    return np.concatenate([X_locations, Y_locations, Z_locations]) 

# time to actually do calibration. 

SECONDS_TO_DETECT = 2 # in seconds

import os, cv2
from tqdm import tqdm 
import numpy as np

ARMFLAPPING_VIDEOS = []
CONTROL_VIDEOS = []
ARMFLAPPING_FPS = [] # store the FPS of all armflapping videos 
CONTROL_FPS = [] # store the FPS of all control videos 

# load these same file names 
import pickle
with open("FILE_NAMES.pkl", 'rb') as f: 
    ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES = pickle.load(f)

for video_name in tqdm(ARMFLAPPING_FILE_NAMES, desc = "armflapping_videos"): 
    try: 
        os.mkdir("behavior_data/armflapping/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/armflapping/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 

        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break  
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image) 

            cv2.imwrite("behavior_data/armflapping/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            i += 1 

        ARMFLAPPING_VIDEOS.append(FRAMES)
        ARMFLAPPING_FPS.append(frame_rate)

    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

for video_name in tqdm(CONTROL_FILE_NAMES, desc = "control_videos"): 
    try:
        os.mkdir("behavior_data/control/" + video_name[1:])
        cap = cv2.VideoCapture('behavior_data/control/' + video_name)  
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) / frame_rate < SECONDS_TO_DETECT: continue # too short! 

        FRAMES = [] # frames for this video 
        i = 0
        while cap.isOpened(): 
            _, image = cap.read() 
            if not _ : 
                break 
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB. 
            FRAMES.append(image)

            cv2.imwrite("behavior_data/control/" + video_name[1:] + "/" + str(i + 1) + ".jpg", image)
            
            i += 1
        CONTROL_VIDEOS.append(FRAMES)
        CONTROL_FPS.append(frame_rate)
        
    except Exception as e: 
        print(e)
        print(f"failed on {video_name}")

ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS = [], []

for FRAMES, file_name in tqdm(zip(CONTROL_VIDEOS, CONTROL_FILE_NAMES)):  
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/control/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    CONTROL_LOCATIONS.append(locs)


# get the locations of all of the videos 

for FRAMES, file_name in tqdm(zip(ARMFLAPPING_VIDEOS,ARMFLAPPING_FILE_NAMES)) :
    locs = []
    for i, frame in enumerate(FRAMES): 
        locs.append(hand_locations(np.array(frame)))

        #FRAME = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #cv2.imwrite("behavior_data/armflapping/" + file_name[1:] + "/" + str(i + 1) + ".jpg", frame)
    ARMFLAPPING_LOCATIONS.append(locs)

N = min([len(locs) for locs in [ARMFLAPPING_LOCATIONS, CONTROL_LOCATIONS]])
ARMFLAPPING_LOCATIONS = ARMFLAPPING_LOCATIONS[:N]
CONTROL_LOCATIONS = CONTROL_LOCATIONS[:N]
ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES[:N])
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES[:N])

# we can create a padding function in order to pad 
def pad(locations, maxlen = 90, padding = "post", truncating = "post"): 
    new_locations = locations.tolist() 
    empty_row = np.zeros((1, 6))
    for i, video in tqdm(enumerate(new_locations)): 
        if len(video) < maxlen:  
            for new_row in range(maxlen - len(video)): 
                if padding == "post": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([new_locations[i], empty_row])
                if padding == "pre": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([empty_row, new_locations[i]])

        if len(video) > maxlen: 
            if truncating == "post": 
                new_locations[i] = new_locations[i][:maxlen]
            elif truncating == "pre": 
                new_locations[i] = new_locations[i][len(video) - maxlen : ]
    return np.array(new_locations)

padded_armflapping_locations = ARMFLAPPING_LOCATIONS
padded_control_locations = CONTROL_LOCATIONS
padded_armflapping_locations = pad(padded_armflapping_locations, maxlen = 90)
padded_control_locations = pad(padded_control_locations, maxlen = 90)
print(padded_control_locations.shape, padded_armflapping_locations.shape)
assert padded_armflapping_locations.shape == padded_control_locations.shape 

ARMFLAPPING_FILE_NAMES = np.array(ARMFLAPPING_FILE_NAMES)
CONTROL_FILE_NAMES = np.array(CONTROL_FILE_NAMES)

"""
# first shuffle 
N = padded_armflapping_locations.shape[0]
perm = np.random.permutation(N)
padded_armflapping_locations = padded_armflapping_locations[perm]
padded_control_locations = padded_control_locations[perm]
ARMFLAPPING_FILE_NAMES = ARMFLAPPING_FILE_NAMES[perm]
CONTROL_FILE_NAMES = CONTROL_FILE_NAMES[perm]
"""

# then balance each fold (still should do this) 
padded_armflapping_locations = np.array_split(padded_armflapping_locations, K)
padded_control_locations = np.array_split(padded_control_locations, K)
ARMFLAPPING_FILE_NAMES = np.array_split(ARMFLAPPING_FILE_NAMES, K)
CONTROL_FILE_NAMES = np.array_split(CONTROL_FILE_NAMES, K)

X_splits = []
y_splits = []

for i in range(K): 
    X_splits.append(np.concatenate([padded_armflapping_locations[i], padded_control_locations[i]]))
    y_splits.append(np.concatenate([np.ones((len(padded_armflapping_locations[i]), 1)), np.zeros((len(padded_control_locations[i]), 1))]))

import pickle

for i, (X_split, y_split, A_FILE_NAMES, C_FILE_NAMES) in enumerate(zip(X_splits, y_splits, ARMFLAPPING_FILE_NAMES, CONTROL_FILE_NAMES)): 
    with open(f"one_point_folds/split{i+1}", 'wb') as f: 
        pickle.dump((X_split, y_split), f)
    

with open(f"one_point_folds_seeds", "wb") as f: 
    pickle.dump((padded_armflapping_locations, ARMFLAPPING_FILE_NAMES, padded_control_locations, CONTROL_FILE_NAMES), f)

control_videos: 100%|██████████| 50/50 [00:24<00:00,  2.07it/s]
50it [01:44,  2.09s/it]
50it [01:52,  2.25s/it]
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_12388/4245297425.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ARMFLAPPING_LOCATIONS = np.array(ARMFLAPPING_LOCATIONS)
/var/folders/37/gk9g67jd759dj9nfbp9jjsyw0000gp/T/ipykernel_12388/4245297425.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  CONTROL_LOCATIONS = np.array(CONTROL_LOCATIONS)
50it [00:00, 12713.86it/s]
50it [00:00, 16041.86it/s]

(50, 90, 6) (50, 90, 6)


In [30]:
def mean_locs(frame): 
    """take in a single frame, and return the mean location"""
    assert len(frame) == 126, len(frame) # make sure this is a vector
    X_locs_1, X_locs_2, Y_locs_1, Y_locs_2, Z_locs_1, Z_locs_2 = frame[:21], frame[21:42], frame[42:63], frame[63:84], frame[84:105], frame[105:]
    return [locs[np.nonzero(locs)].mean() for locs in [X_locs_1, X_locs_2, Y_locs_1, Y_locs_2, Z_locs_1, Z_locs_2] ]

new_X = [] 
for video in range(X.shape[0]): 
    matrix_video = X[video]
    new_X.append(np.apply_along_axis(mean_locs, 1, matrix_video))
    
X = np.array(new_X, copy = True) 
assert len(X.shape) == 3

splits = [] # stores k (X_i, y_i) splits
X_splits, y_splits = np.array_split(X, K), np.array_split(y, K)
for X_split, y_split in zip(X_splits, y_splits): 
    splits.append((X_split, y_split))

for i, split in enumerate(splits): 
    with open(f"mean_point_folds/split{i+1}", 'wb') as f: 
        pickle.dump(split, f)

<ipython-input-30-cf37e4d2f41e>:5: RuntimeWarning: Mean of empty slice.
  return [locs[np.nonzero(locs)].mean() for locs in [X_locs_1, X_locs_2, Y_locs_1, Y_locs_2, Z_locs_1, Z_locs_2] ]
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [63]:
with open("temp_X.pkl", 'wb') as f: 
    pickle.dump(X, f)

with open("temp_X.pkl", 'rb') as f: 
    X = pickle.load(f)

In [27]:
X = np.nan_to_num(X)
splits = [] # stores k (X_i, y_i) splits
X_splits, y_splits = np.array_split(X, K), np.array_split(y, K)
for X_split, y_split in zip(X_splits, y_splits): 
    splits.append((X_split, y_split))

for i, split in enumerate(splits): 
    with open(f"mean_point_folds/split{i+1}", 'wb') as f: 
        pickle.dump(split, f)

In [4]:
for file in os.listdir("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control"):
         if os.path.isfile("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control/" + file):
             continue
         else:
             #shutil.rmtree("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/armflapping/" + file)
             for vid_name in os.listdir("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control/" + file):
                 image = cv2.imread("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control/" + file + "/" + vid_name)
                 image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                 os.remove("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control/" + file + "/" + vid_name)
                 cv2.imwrite("/Users/anish/Documents/Machine Learning Env/ActivRecognition-Autism-Diagnosis/behavior_data/control/" + file + "/" + vid_name, image)

In [3]:
import cv2, os